In [66]:
import pandas as pd
import warnings
import re
import tables
from datetime import datetime
warnings.filterwarnings('ignore')

In [67]:
def cria_dataframe_base(animais_sem_id, df_pedigree):
    df_animais_sem_id_completo = pd.merge(animais_sem_id, df_pedigree, on=['IDANIMAL'], how='left', indicator=True, suffixes=('','_drop'))
    df_animais_sem_id_completo = df_animais_sem_id_completo.drop(df_animais_sem_id_completo.filter(regex='_drop$').columns, axis=1)
    df_animais_sem_id_completo = df_animais_sem_id_completo[df_animais_sem_id_completo['_merge']=='both']
    df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates(subset='IDANIMAL')
    return df_animais_sem_id_completo

In [68]:
def cria_id_produto(df):
    if 'id_produto' in df.columns:
        df['id_produto'] = df['id_produto'].fillna('GIRO' + df['IDANIMAL'].astype(str))
        if '_merge' in df.columns:
            df = df.drop(columns='_merge')
    else:
        df = df.assign(id_produto=None)
        df['id_produto'] = df['id_produto'].fillna('GIRO' + df['IDANIMAL'].astype(str))
    return df

In [69]:
def busca_id_pai_mae_na_genealogia_embrapa(df):
    df = pd.merge(df, df[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
    df = df.drop_duplicates()
    df = pd.merge(df, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
    df = df.drop_duplicates()
    if 'id_touro' in df.columns:
        df = df.drop(columns='id_touro')
    if 'id_vaca' in df.columns:
        df = df.drop(columns='id_vaca')
    df = df.rename(columns={'id_produto_drop':'id_touro'})
    df = df.rename(columns={'id_produto_tes':'id_vaca'})
    df = df.drop(columns='IDANIMAL_drop')
    df = df.drop(columns='IDANIMAL_tes')
    #df = df.drop(columns='_merge')
    return df

In [70]:
def busca_id_touro_e_id_vaca_pelo_cod_prod_genealogia_embrapa(df):
    if '_merge' in df.columns:
        df = df.drop(columns='_merge')
    df_id_touro = pd.merge(df, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
    #ID_TOURO
    df = pd.merge(df, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
    df = df.drop_duplicates()
    df['id_touro'].fillna(df['id_produto_drop'], inplace=True)
    df = df.drop(columns='id_produto_drop')

    #ID_VACA
    df_id_vaca = pd.merge(df, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
    df = pd.merge(df, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
    df = df.drop_duplicates()
    df['id_vaca'].fillna(df['id_produto_drop'], inplace=True)
    df = df.drop(columns='id_produto_drop')
    return df

In [71]:
def pega_id_produto_genealogia_embrapa_registro_3_letras_e_GSPAI_GSMAE_11(df):
    df['RGMAE'] = df['RGMAE'].astype(str)
    df['RGPAI'] = df['RGPAI'].astype(str)
    df_sub_tabela_gir = df[df['GSPAI']==11]
    df_sub_tabela_gir = df[df['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_sub_tabela_gir1 = df[df['GSMAE']==11]
    df_sub_tabela_gir1 = df[df['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_sub_touros_id = pd.merge(df_sub_tabela_gir, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGPAI'], right_on=['registro'])
    df_sub_vacas_id = pd.merge(df_sub_tabela_gir1, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGMAE'], right_on=['registro'])

    df = pd.merge(df, df_sub_touros_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_touro'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')
    df = pd.merge(df, df_sub_vacas_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_vaca'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')
    return df

In [72]:
def gera_lista_animais_pelo_id_produto_touros_vacas_genealogia_RG_menor_3(df):
    df = df.drop_duplicates()
    df = df.loc[~df.index.duplicated(keep='first')]
    df.reset_index(inplace=True, drop=True)
    df_sub_tabela_gir1 = df[df['GSPAI']==11]
    df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    #Pegando os 4 primeiros caracteres do nome
    df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['NOMEPAI'].str.slice(start=0, stop=4)
    df_sub_tabela_gir1 = df_sub_tabela_gir1.drop_duplicates()
    df_sub_tabela_gir1.reset_index(inplace=True, drop=True)

    df_subtabela_genealogia = df_genealogia_embrapa
    df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
    df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)
    df_subtabela_genealogia = df_subtabela_genealogia.drop_duplicates()

    dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])
    dj_juntando_tabelas = dj_juntando_tabelas.rename(columns={'id_produto_y':'id_touro_encontrado'})
    dj_juntando_tabelas = dj_juntando_tabelas.reset_index(drop=True) # Reset index
    dj_juntando_tabelas.reset_index(inplace=True, drop=True)

    df_sub_tabela_gir2 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
    df_sub_tabela_gir2 = df_sub_tabela_gir2[~df_sub_tabela_gir2['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    #Pegando os 4 primeiros caracteres do nome
    df_sub_tabela_gir2['nome4caracteres'] = df_sub_tabela_gir2['NOMEMAE'].str.slice(start=0, stop=4)
    df_sub_tabela_gir2.reset_index(inplace=True, drop=True)

    df_subtabela_genealogia1 = df_genealogia_embrapa
    df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)
    df_subtabela_genealogia1 = df_subtabela_genealogia1.drop_duplicates()
    df_subtabela_genealogia1.reset_index(inplace=True, drop=True)

    dj_juntando_tabelas1 = pd.merge(df_sub_tabela_gir2, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])
    dj_juntando_tabelas1 = dj_juntando_tabelas1.rename(columns={'id_produto_y':'id_vaca_encontrado'})
    dj_juntando_tabelas1 = dj_juntando_tabelas1.reset_index(drop=True) # Reset index
    dj_juntando_tabelas1.reset_index(inplace=True, drop=True)
    dj_juntando_tabelas = dj_juntando_tabelas.drop_duplicates()
    dj_juntando_tabelas1 = dj_juntando_tabelas1.drop_duplicates()

    dj_juntando_tabelas = dj_juntando_tabelas.reset_index(drop=True) # Reset index
    dj_juntando_tabelas1 = dj_juntando_tabelas1.reset_index(drop=True) # Reset index

    tes = pd.concat([dj_juntando_tabelas,dj_juntando_tabelas1], axis =1)
    tes= tes.drop_duplicates()
    return tes


In [73]:
def adiciona_animais_rg_menor_3_no_dataframe_principal(df, lista):
    df = pd.merge(df, lista[['IDANIMAL', 'id_vaca_encontrado', 'id_touro_encontrado']], on=['IDANIMAL'], how='left')
    df['id_touro'].fillna(df['id_touro_encontrado'], inplace=True)
    df = df.drop(columns='id_touro_encontrado')
    df['id_vaca'].fillna(df['id_vaca_encontrado'], inplace=True)
    df = df.drop(columns='id_vaca_encontrado')
    return df

In [74]:
def formata_df_no_modelo_genealogia_embrapa(df):
    #trata o dnasc
    if 'DNASC' in df.columns:
        df = df.rename(columns={'DNASC':'nasc'})
    #trata o formata_df_no_modelo_genealogia_embrapa
    if 'NOMEVACA' in df.columns:
        df = df.rename(columns={'NOMEVACA':'nome'})
    # Adiciona coluna "id"
    df = df.assign(id=None)
    #Atribui valores de id_produto a id
    df['id'] = df['id_produto']
    # Adiciona coluna "abcz" com valores iguais a null
    df = df.assign(abcz=None)
    #renomeia RGVACA para registro
    df = df.rename(columns={'RGVACA':'registro'})
    #renomeia nomea para nome
    df = df.rename(columns={'nomea':'nome'})
    #renomeia SEXO para sexo
    df = df.rename(columns={'SEXO':'sexo'})
    # Adiciona coluna "paternidade" com valores iguais a null
    df = df.assign(paternidade=None)
    # Adiciona coluna "ck_sire" com valores iguais a null
    df = df.assign(ck_sire=None)
    # Adiciona coluna "ck_dam" com valores iguais a null
    df = df.assign(ck_dam=None)
    # Adiciona coluna "gen_valido" com valores iguais a null
    df = df.assign(gen_valido=None)
    # Adiciona coluna "nasc_est"
    df = df.assign(nasc_est=None)
    #Atribui os valores de nasc a nasc_est
    df['nasc_est'] = df['nasc']
    # Adiciona coluna "gr_psg"
    df = df.assign(gr_psg=None)
    # Adiciona coluna "gr_ps"
    df = df.assign(gr_ps=None)
    # Adiciona coluna "gr_tp"
    df = df.assign(gr_tp=None)
    # Adiciona coluna "rebc2_cri"
    df = df.assign(rebc_cri=None)
    #renomeia rebc_or para rebc_pr
    df = df.rename(columns={'rebc_or':'rebc_pr'})
    # Adiciona coluna "cat"
    if 'cat' in df.columns:
        pass
    else:
        df = df.assign(cat=None)
    # Adiciona coluna "ficha_a"
    df = df.assign(ficha_a=None)
    #renomeia IDANIMAL para idanimal
    df = df.rename(columns={'IDANIMAL':'idanimal'})
    # Adiciona coluna "origem"
    df = df.assign(origem='GIRO')
    # Adiciona coluna "data_insercao"
    df = df.assign(data_insercao=None)
    data_atual = datetime.today().strftime('%Y-%m-%d')
    df['data_insercao'] = data_atual
    df = df[['id_produto', 'id', 'abcz', 'registro','nome','sexo','paternidade','ck_sire','id_touro','id_vaca','ck_dam','gen_valido','nasc','nasc_est','cgen_a','gs','gr_psg','gr_ps','gr_tp','rebc_cri','rebc_pr','cat','ficha_a','cod_prod','idanimal','origem', 'data_insercao']]
    return df


In [75]:
def gera_lista_de_animais_com_id_vaca_touro(df):
    df = df.drop_duplicates()
    animais_com_ids = df[~df['id_vaca'].isnull() & ~df['id_touro'].isnull()]
    return animais_com_ids

In [76]:
def gera_lista_de_animais_sem_id_vaca_e_ou_sem_id_touro(animais_sem_id_completo):
    animais_sem_id_vaca = animais_sem_id_completo[animais_sem_id_completo['id_vaca'].isnull()]
    animais_sem_id_vaca = animais_sem_id_vaca.drop_duplicates()
    animais_sem_id_touro = animais_sem_id_completo[animais_sem_id_completo['id_touro'].isnull()]
    animais_sem_id_touro = animais_sem_id_touro.drop_duplicates()
    animais_sem_id = pd.concat([animais_sem_id_touro, animais_sem_id_vaca])
    animais_sem_id = animais_sem_id.drop_duplicates()


    return animais_sem_id

In [77]:
def adiciona_colunas_codprod_codpai_cod_mae(df, genealogia_abcz):
  sub_tabela_gs_0 = df[df['gs'] == 0]
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  df_temporaria1 = pd.merge(sub_tabela_gs_0, genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
  sub_tabela_gs_0=df_temporaria1
  sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()

  df_temporaria4 = pd.merge(df, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
  df=df_temporaria4
  df = df.drop_duplicates()
  return df

In [78]:
def pegar_codpai_codmae_abcz_rg_3_letras(df, genealogia_abcz):
  if '_merge' in df.columns:
      df = df.drop(columns='_merge')
  if '_merge' in genealogia_abcz.columns:
      df = df.drop(columns='_merge')

  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela para maes
  subtabela_mae = df[df['GSMAE'] == 11]
  #e o cod_mae e nulo
  subtabela_mae
  # selecionar apenas as linhas que começam com letras
  subtabela_mae = subtabela_mae[subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Buscando cod_mae na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna cod_prod_mae no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela em que gspai = 11
  subtabela_pai = df[df['GSPAI'] == 11]

  # selecionar apenas as linhas que começam com letras
  subtabela_pai = subtabela_pai[subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na ABCZ
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Criando coluna do cod_prod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando valores da cod_pai e cod_mae do df com os valores das colunas inseridas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')

  return df


In [79]:
def pegar_codpai_codmae_abcz_rg_diferente_3_letras(df, genealogia_abcz):
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela
  subtabela_pai = df[df['GSPAI'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_pai = subtabela_pai[~subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_pai_gen'})
  subtabela_pai = subtabela_pai.drop_duplicates()
  genealogia_abcz = genealogia_abcz.drop_duplicates()
  subtabela_pai['nome4caracteres'] = subtabela_pai['NOMEPAI'].str.slice(start=0, stop=4)
  genealogia_abcz['nome4caracteres'] = genealogia_abcz['nome'].str.slice(start=0, stop=4)
  genealogia_abcz_pai = genealogia_abcz
  genealogia_abcz_pai = genealogia_abcz[genealogia_abcz['sexo']== 'M']
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz_pai[['RGVACA', 'nome4caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela mae
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = df[df['GSMAE'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_mae = subtabela_mae[~subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_mae na abcz
  subtabela_mae['nome4caracteres'] = subtabela_mae['NOMEMAE'].str.slice(start=0, stop=4)
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
  #criando coluna com os 10 caracteres do nome
  genealogia_abcz_mae = genealogia_abcz
  genealogia_abcz_mae = genealogia_abcz[genealogia_abcz['sexo']== 'F']
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz_mae[['RGVACA','nome4caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_mae_gen no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando cod_pai e cod_mae com base nas novas colunas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')
  return df

# Lendo planilhas

In [80]:
df_animais_sem_id = pd.read_hdf('df_animais_sem_id.h5', key='df')
df_genealogia_embrapa = pd.read_hdf('df_genealogia_embrapa.h5', key='df')
df_fazendas = pd.read_hdf('df_fazendas.h5', key='df')
df_criadores = pd.read_hdf('df_criadores.h5', key='df')
df_genealogia_abcz = pd.read_hdf('df_genealogia_abcz.h5', key='df')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')

# Imprimindo dataframes

In [81]:
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_x')
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_y')

In [82]:
df_animais_sem_id_completo = cria_dataframe_base(df_animais_sem_id, pedigree)

In [83]:
cria_id_produto(df_animais_sem_id_completo)
df_animais_sem_id_completo

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nome10caracteres,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,_merge
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,NANAIMPERA,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,both
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,LANAAFIV,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,both
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,EDILAINEIM,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,both
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,ARMAMERIDI,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,both
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,CELESTE136,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,JPZQUIRERA,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,both
563,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,DALIAIFIV,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,both
564,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,QUINTAICO,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,both
565,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,LIADASARA,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,both


In [84]:
df_animais_sem_id_completo = busca_id_pai_mae_na_genealogia_embrapa(df_animais_sem_id_completo)
df_animais_sem_id_completo

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,_merge,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,both,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,both,NaN,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,both,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,both,NaN,JNR167
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,both,NaN,GIRO1086826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,both,NaN,NaN
555,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,both,NaN,JMCH76
556,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,both,NaN,GIRO1323923
557,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,both,NaN,NaN


In [85]:
df_animais_sem_id_completo = busca_id_touro_e_id_vaca_pelo_cod_prod_genealogia_embrapa(df_animais_sem_id_completo)
df_animais_sem_id_completo

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN,ACFG1840
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,NaN,JNR167
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,GIRO1086826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,NaN,JPZ64
555,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,NaN,JMCH76
556,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,NaN,GIRO1323923
557,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,NaN,NaN


In [86]:
df_animais_sem_id_completo = pega_id_produto_genealogia_embrapa_registro_3_letras_e_GSPAI_GSMAE_11(df_animais_sem_id_completo)
df_animais_sem_id_completo

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN,ACFG1840
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,NaN,JNR167
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,GIRO1086826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,NaN,JPZ64
555,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,NaN,JMCH76
556,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,NaN,GIRO1323923
557,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,NaN,NaN


In [87]:
lista = gera_lista_animais_pelo_id_produto_touros_vacas_genealogia_RG_menor_3(df_animais_sem_id_completo)
lista
#Conferencia Manual

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,nome4caracteres,id_vaca_encontrado,registro
0,5191,MENINA MODELO 2K,12,4833-AR,MODELO TE DE BRASILIA,11,B5213,FEITICEIRA 2K,10,C-1072,...,F,SANDY-VALLEY BOLTON 2 ETN,APOSTA,1/2HOL 1/2GIR,519.0,NaN,618828,APOS,618828,E1438
1,5191,MELISSA MODELO 2K,12,5416-AR,MODELO TE DE BRASILIA,11,B5213,IOGA 2K,10,C-1073,...,F,DE-SU GILLESPY-ET,APOSTA,1/2HOL 1/2GIR,519.0,NaN,618828,APOS,618828,E1438
2,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,F,EDG SS SLADE 8314-ET,PL ZARA NAPOLITANO,1/2HOL 1/2GIR,519.0,NaN,155945,PL Z,155945,F6406
3,5407,GARFIEIRA,11,PVBG1140,RADAR DOS POCOES,11,A7368,LARA FIV DA SADONANA,11,SDNA34,...,F,CROTEAU LESPERRON UNIX,VAIDOSA F.INDIANA,1/2HOL 1/2GIR,519.0,NaN,GIRO1179905,VAID,GIRO1179905,G726
4,5407,FORTUNA DA CRISTAL,11,PVBG855,RADAR DOS POCOES,11,A7368,RADULA TE DO GAVIAO,11,GAV980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8030,VENTOSA CADARSO FZD,14,4241-AQ,CADARSO C-054,11,B32,SAVANA PLANET FIV FZD,12,4961-U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#df_animais_sem_id_completo = adiciona_animais_rg_menor_3_no_dataframe_principal(df_animais_sem_id_completo, lista)
df_animais_sem_id_completo['IDANIMAL'].value_counts()

2038826    2
1631099    1
2498996    1
2677677    1
2098291    1
          ..
2678433    1
1539669    1
2522343    1
2499111    1
1838611    1
Name: IDANIMAL, Length: 558, dtype: int64

In [94]:
df_animais_sem_id_completo.loc[df_animais_sem_id_completo['IDANIMAL'] == 2038826]

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
56,2381,FERTILIDADE ALTACEO JM NOVO HO,58,4899-AZ,RICKLAND ALTACEO-ET,10,AX138598,BALELA YANK JM NOVO HORIZONTE,38,3619-U,...,2017,FERTILIDADE ALTACEO JM NOVO HORIZONTE,01/11/2017,F,RICKLAND ALTACEO-ET,BALELA YANK JM NOVO HORIZONTE,5/8HOL 3/8GIR,693,NaN,206820
57,2381,FERTILIDADE ALTACEO JM NOVO HO,58,4899-AZ,RICKLAND ALTACEO-ET,10,AX138598,BALELA YANK JM NOVO HORIZONTE,38,3619-U,...,2017,FERTILIDADE ALTACEO JM NOVO HORIZONTE,01/11/2017,F,RICKLAND ALTACEO-ET,BALELA YANK JM NOVO HORIZONTE,5/8HOL 3/8GIR,693,NaN,GIRO1398001


In [ ]:
lista_de_animais_com_id_vaca_touro = gera_lista_de_animais_com_id_vaca_touro(df_animais_sem_id_completo)
lista_de_animais_com_id_vaca_touro

In [ ]:
lista_de_animais_sem_id_vaca_touro = gera_lista_de_animais_sem_id_vaca_e_ou_sem_id_touro(df_animais_sem_id_completo)
lista_de_animais_sem_id_vaca_touro

1 - Criar id_touro (GIRO + id_pai)
2 - juntar animais sem id com pedigree girolando (id_pai com IDANIMAL)
3 - Juntar com genealogia_embrapa (IDANIMAL com IDANIMAL) - se der match ta pronto (adicionar aos animais que estao finalizados para o Pedigree) - add coluna geracao
4 - Executar todas as funcoes exceto a primeira e segunda para preencher id_touro e id_vaca
5 - resultado de  gera_lista_de_animais_com_id_vaca_touro sao os animais finalizados e prontos para o banco
6 - pegar o resultado de gera_lista_de_animais_sem_id_vaca_e_ou_sem_id_touro e voltar para o passo 1
5 - id_pai = IDANIMAL e id_produto = id_touro

1 - Criar id_touro (GIRO + id_pai)

In [ ]:
lista_de_animais_sem_id_vaca_touro['id_touro'] = lista_de_animais_sem_id_vaca_touro['id_touro'].fillna('GIRO' + lista_de_animais_sem_id_vaca_touro['IDPAI'].astype(str))
lista_de_animais_sem_id_vaca_touro['id_vaca'] = lista_de_animais_sem_id_vaca_touro['id_vaca'].fillna('GIRO' + lista_de_animais_sem_id_vaca_touro['IDMAE'].astype(str))

2 - juntar animais sem id com pedigree girolando (id_pai com IDANIMAL)

In [ ]:
df_juntando_pai = pd.merge(lista_de_animais_sem_id_vaca_touro[['IDPAI']] , pedigree, left_on=['IDPAI'], right_on=['IDANIMAL'])
df_juntando_pai = df_juntando_pai.drop(columns ='IDPAI_x')
df_juntando_pai = df_juntando_pai.rename(columns={'IDPAI_y':'IDPAI'})
df_juntando_pai = df_juntando_pai.drop_duplicates()
df_juntando_mae = pd.merge(lista_de_animais_sem_id_vaca_touro[['IDMAE']] , pedigree, left_on=['IDMAE'], right_on=['IDANIMAL'])
df_juntando_mae = df_juntando_mae.drop(columns='IDMAE_x')
df_juntando_mae = df_juntando_mae.rename(columns={'IDMAE_y':'IDMAE'})
df_juntando_mae = df_juntando_mae.drop_duplicates()
df_pais_a_serem_add = pd.concat([df_juntando_pai, df_juntando_mae])
df_pais_a_serem_add = df_pais_a_serem_add.drop_duplicates(subset='IDANIMAL')
df_pais_a_serem_add

3 - Juntar com genealogia_embrapa (IDANIMAL com IDANIMAL) - se der match ta pronto (adicionar aos animais que estao finalizados para o Pedigree) - add coluna geracao

In [ ]:
df_match_genealogia = pd.merge(df_pais_a_serem_add, df_genealogia_embrapa, on=['IDANIMAL'], suffixes=('', '_drop'), how='left', indicator=True)
lista_de_animais_2_ger_prontos = df_match_genealogia[df_match_genealogia['_merge']=='both']
lista_de_animais_nao_encontrados = df_match_genealogia[df_match_genealogia['_merge']=='left_only']

In [ ]:
lista_de_animais_2_ger_prontos

In [ ]:
lista_de_animais_nao_encontrados

Adicionar cod_prod cod_pai e cod_mae

In [ ]:
lista_de_animais_nao_encontrados =adiciona_colunas_codprod_codpai_cod_mae(lista_de_animais_nao_encontrados, df_genealogia_abcz)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_nao_encontrados = pegar_codpai_codmae_abcz_rg_3_letras(lista_de_animais_nao_encontrados, df_genealogia_abcz)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_nao_encontrados = pegar_codpai_codmae_abcz_rg_diferente_3_letras(lista_de_animais_nao_encontrados, df_genealogia_abcz)
lista_de_animais_nao_encontrados

4 - Executar todas as funcoes exceto a primeira e segunda para preencher id_touro e id_vaca

In [ ]:
lista_de_animais_nao_encontrados = cria_id_produto(lista_de_animais_nao_encontrados)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_nao_encontrados = busca_id_pai_mae_na_genealogia_embrapa(lista_de_animais_nao_encontrados)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_nao_encontrados = busca_id_touro_e_id_vaca_pelo_cod_prod_genealogia_embrapa(lista_de_animais_nao_encontrados)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_nao_encontrados = pega_id_produto_genealogia_embrapa_registro_3_letras_e_GSPAI_GSMAE_11(lista_de_animais_nao_encontrados)
lista_de_animais_nao_encontrados

In [ ]:
listat = gera_lista_animais_pelo_id_produto_touros_vacas_genealogia_RG_menor_3(lista_de_animais_nao_encontrados)
listat

In [ ]:
lista_de_animais_nao_encontrados = adiciona_animais_rg_menor_3_no_dataframe_principal(lista_de_animais_nao_encontrados, lista)
lista_de_animais_nao_encontrados

In [ ]:
lista_de_animais_com_id_vaca_touro_1 = gera_lista_de_animais_com_id_vaca_touro(lista_de_animais_nao_encontrados)
lista_de_animais_com_id_vaca_touro_1

In [ ]:
lista_de_animais_nao_encontrados.columns

In [ ]:
lista_de_animais_sem_id_vaca_touro_1 = gera_lista_de_animais_sem_id_vaca_e_ou_sem_id_touro(lista_de_animais_nao_encontrados)
lista_de_animais_sem_id_vaca_touro_1

df_animais_com_ids_abcz = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids_abcz)
df_animais_com_ids = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids)
df_animais_com_ids = pd.concat([df_animais_com_ids, df_animais_com_ids_abcz])
df_animais_com_ids